In [1]:
import numpy as np
import pandas as pd

In [2]:
data=pd.read_csv('/content/spam.csv',encoding='latin1')

In [3]:
data

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [4]:
data=data.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1)

In [5]:
data.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
data=data.rename(columns={'v1':'Labels','v2':'Email'})

In [7]:
data.head()

,Labels,Email
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [8]:
len(list(set(data['Email'])))

5169

In [9]:
data['Labels'].value_counts()

ham     4825
spam     747
Name: Labels, dtype: int64

Lets do preprocessing

In [10]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
import re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [11]:
wnl=WordNetLemmatizer()
corpus=[]

In [12]:
for i in range(len(data)):
  review=re.sub('[^a-zA-Z]',' ',data['Email'][i])
  review=review.lower()
  review=review.split()
  review=[wnl.lemmatize(word) for word in review if not word in stopwords.words('english')]
  review=' '.join(review)
  corpus.append(review)

In [13]:
data['Clean_emails']=corpus

In [14]:
data.head()

,Labels,Email,Clean_emails
0,ham,"Go until jurong point, crazy.. Available only ...",go jurong point crazy available bugis n great ...
1,ham,Ok lar... Joking wif u oni...,ok lar joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry wkly comp win fa cup final tkts st ...
3,ham,U dun say so early hor... U c already then say...,u dun say early hor u c already say
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah think go usf life around though


In [15]:
maxl=max([len(s) for s in data['Clean_emails']])
maxl

442

In [16]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [17]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(data['Clean_emails'])
sequence=tokenizer.texts_to_sequences(data['Clean_emails'])
padded_sequences=pad_sequences(sequence)

In [18]:
from sklearn.preprocessing import LabelEncoder

In [19]:
le=LabelEncoder()
le.fit(data['Labels'])
y=le.transform(data['Labels'])

In [20]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM,Bidirectional
from keras.layers import SpatialDropout1D
from keras.layers.embeddings import Embedding
from keras.utils import to_categorical

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
X_train,X_test,y_train,y_test=train_test_split(padded_sequences,y,test_size=0.3,random_state=42)

In [23]:
y_train.shape

(3900,)

In [24]:
y_trains=to_categorical(y_train)
y_trains=np.array(y_trains)

In [25]:
y_tests=to_categorical(y_test)
y_tests=np.array(y_tests)

In [26]:
y_trains.shape

(3900, 2)

In [27]:
model=Sequential()
model.add(Embedding(input_dim=10000,output_dim=200,input_length=maxl))
model.add(SpatialDropout1D(0.2))
model.add(Bidirectional(LSTM(150, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(2, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 442, 200)          2000000   
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 442, 200)          0         
_________________________________________________________________
bidirectional (Bidirectional (None, 300)               421200    
_________________________________________________________________
dense (Dense)                (None, 2)                 602       
Total params: 2,421,802
Trainable params: 2,421,802
Non-trainable params: 0
_________________________________________________________________
None


In [28]:
model.fit(X_train,y_trains,batch_size=64,epochs=10,validation_data=(X_test,y_tests))

Epoch 1/10
61/61 [==============================] - 61s 912ms/step - loss: 0.3948 - accuracy: 0.8187 - val_loss: 0.0727 - val_accuracy: 0.9821
Epoch 2/10
61/61 [==============================] - 55s 896ms/step - loss: 0.0448 - accuracy: 0.9893 - val_loss: 0.0671 - val_accuracy: 0.9821
Epoch 3/10
61/61 [==============================] - 55s 896ms/step - loss: 0.0149 - accuracy: 0.9967 - val_loss: 0.0513 - val_accuracy: 0.9880
Epoch 4/10
61/61 [==============================] - 55s 900ms/step - loss: 0.0125 - accuracy: 0.9981 - val_loss: 0.0572 - val_accuracy: 0.9868
Epoch 5/10
61/61 [==============================] - 55s 899ms/step - loss: 0.0057 - accuracy: 0.9985 - val_loss: 0.0586 - val_accuracy: 0.9874
Epoch 6/10
61/61 [==============================] - 55s 902ms/step - loss: 0.0026 - accuracy: 0.9997 - val_loss: 0.0671 - val_accuracy: 0.9868
Epoch 7/10
61/61 [==============================] - 55s 903ms/step - loss: 0.0013 - accuracy: 0.9997 - val_loss: 0.0663 - val_accuracy: 0.9880

In [29]:
labels=['HAM','SPAM']
text=['free entry win lot of money by clicking this link']
seq=tokenizer.texts_to_sequences(text)
padded=pad_sequences(seq,maxlen=maxl)
pred=model.predict(padded)
print("Probability distribution: ", pred)
print("Is this a HAM or SPAM ? ")
print(labels[np.argmax(pred)])

Probability distribution:  [[1.2952487e-06 9.9999869e-01]]
Is this a HAM or SPAM ? 
SPAM


In [30]:
labels=['HAM','SPAM']
text=['hello satyam i was busy please resend me that email']
seq=tokenizer.texts_to_sequences(text)
padded=pad_sequences(seq)
pred=model.predict(padded)
print('Probability distribution-',pred)
print('Is this a HAM or SPAM?')
print(labels[np.argmax(pred)])

Probability distribution- [[9.9996531e-01 3.4737426e-05]]
Is this a HAM or SPAM?
HAM
